In [45]:
# A_Prep_Filter_Non_Encroaching_blue_flag_bb.jpynb 
# Process images and associated annotation files in a job folder and filter out specific cases where a bounding box 
# of a given class (e.g., red or red-yellow flag) does not overlap with or contain a blue flag. 
# Find images where the red flag bounding box does not contain any portion of the blue flag
# Searches through a folder of datasets; examine each image and its annotation in the obj_train_data sub-folder, 
# checks whether the class bounding box contains a strong contrast leading into a darker blue region.
# If no such area is found, copy the image to a specified output folder.

## Could improve by allowing an edge of e.g.3 pixels as in image 1918, 1920,  in series Job_23
##  1921 ha tiny area 1945 etc. 

# 6. Check for Blue Flags
# Calls the contains_blue_flag function to determine if the ROI contains a blue flag:
# Converts the ROI to HSV color space.
# Uses a mask to detect pixels in the blue color range.
# Detects edges in the masked area using cv2.Canny.
# If any edges are found, the script assumes a blue flag is present.
# Helper Function: contains_blue_flag

# Converts an image's ROI to HSV color space to separate colors by hue.
# Masks areas within a specific blue color range (using lower and upper thresholds).
# Uses cv2.Canny to detect edges in the blue mask.
# Purpose: To ensure that only images where bounding boxes of class class_id (e.g., red or red-yellow flags) do not encroach on blue flags are retained for further processing.
# Output: A filtered dataset of images and annotations in the _a folder.

import os
import cv2
import shutil
import numpy as np

def filter_non_encroaching_blue_flag_bb(base_folder, job, class_id):
    """
    In job folders, find images where class 0  or 1 bounding box does not contain/overlap with a blue flag.
    :param base_folder: Path to the folder containing job folders.
    :param class_id: Class ID for which bounding box is checked.
    """
    print(f"Checking {job} for flags where the blue flag bounding box does not overlap the red or red yellow flag in ...")
    print(f"Base folder {base_folder}...")

    #for job_folder in os.listdir(base_folder):
    suffix = "_filter"
    new_folder = job + suffix
        
    job_path = os.path.join(base_folder, job)
    output_folder= os.path.join(base_folder, new_folder) # ... reduced/job_n_a
    obj_train_new_data_path = os.path.join(output_folder, "obj_train_data") #reduced/job_n_a/obj_train_data
    obj_train_data_path = os.path.join(job_path, "obj_train_data")

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    #print(f"Checking {job} for non overlapping flags...")

    if not os.path.exists(obj_train_new_data_path):
        os.makedirs(obj_train_new_data_path)
         
    if not os.path.exists(obj_train_data_path):
        print(f"No obj_train_data folder in {job}, skipping...")
        return
        
    for file in os.listdir(obj_train_data_path):
        if file.lower().endswith(".txt"):  # Annotation file
            annotation_path = os.path.join(obj_train_data_path, file)
            image_path = os.path.join(obj_train_data_path, file.rsplit('.', 1)[0] + ".PNG")
            #images = [f for f in os.listdir(obj_train_data_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
            if not os.path.exists(image_path):
                print(f"Image file {image_path} not found, skipping...")
                continue
                
            # Read annotation
            with open(annotation_path, 'r') as f:
                annotations = f.readlines()
                
            # Check for class 0 bounding boxes
            for line in annotations:
                elements = line.strip().split()
                if int(elements[0]) == class_id:
                    # Extract bounding box details
                    x_center, y_center, width, height = map(float, elements[1:])          
                    # Convert normalized coordinates to pixel values
                    image = cv2.imread(image_path)
                    img_height, img_width = image.shape[:2]
                    x1 = int((x_center - width / 2) * img_width)
                    y1 = int((y_center - height / 2) * img_height)
                    x2 = int((x_center + width / 2) * img_width)
                    y2 = int((y_center + height / 2) * img_height)
                        
                    # Extract the region of interest (ROI)
                    roi = image[y1:y2, x1:x2]

                    #if contains_blue_flag(roi):
                        # print(f"Contains blue flag? {image_path} ")
                        
                    # Check for strong contrast into darker blue
                    if not contains_blue_flag(roi):
                      #  if not os.path.exists(output_folder):
                      #  os.makedirs(output_folder)
                        #print(f"Beginning filtering {job} for non overlapping flags...")
    
                        # Move image and annotation to output folder
                        # shutil.copy(image_path, os.path.join(output_folder, os.path.basename(image_path)))
                        shutil.move(image_path, os.path.join(obj_train_new_data_path, os.path.basename(image_path)))
                        shutil.move(annotation_path, os.path.join(obj_train_new_data_path, os.path.basename(annotation_path)))
                        #shutil.move(image_path, os.path.join(obj_train_new_data_path, os.path.basename(image_path)))
                        #shutil.move(annotation_path, os.path.join(obj_train_new_data_path, os.path.basename(annotation_path)))
                        # print(f"Moved {os.path.basename(image_path)} to {output_folder}")
                        break
    print(f"Finished filtering {job}, ...")

def filter_non_encroaching_blue_flag_bb_flat(base_folder, job, class_id):
    """
    In job folders, find images where class 0  or 1 bounding box does not contain/overlap with a blue flag.
    :param base_folder: Path to the folder containing job folders.
    :param class_id: Class ID for which bounding box is checked.
    """
    print(f"Checking {job} for flags where the blue flag bounding box does not overlap the red or red yellow flag in ...")
    print(f"Base folder {base_folder}...")

    #for job_folder in os.listdir(base_folder):
    suffix = "_filter"
    new_folder = job + suffix
        
    job_path = os.path.join(base_folder, job)
    output_folder= os.path.join(base_folder, new_folder) # ... reduced/job_n_a
    obj_train_new_data_path = output_folder #os.path.join(output_folder, "obj_train_data") #reduced/job_n_a/obj_train_data
    obj_train_data_path = job_path #os.path.join(job_path, "obj_train_data")

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    #print(f"Checking {job} for non overlapping flags...")

    if not os.path.exists(obj_train_new_data_path):
        os.makedirs(obj_train_new_data_path)
         
    if not os.path.exists(obj_train_data_path):
        print(f"No obj_train_data folder in {job}, skipping...")
        return
        
    for file in os.listdir(obj_train_data_path):
        if file.lower().endswith(".txt"):  # Annotation file
            annotation_path = os.path.join(obj_train_data_path, file)
            image_path = os.path.join(obj_train_data_path, file.rsplit('.', 1)[0] + ".PNG")
            #images = [f for f in os.listdir(obj_train_data_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
            if not os.path.exists(image_path):
                print(f"Image file {image_path} not found, skipping...")
                continue
                
            # Read annotation
            with open(annotation_path, 'r') as f:
                annotations = f.readlines()
                
            # Check for class 0 bounding boxes
            for line in annotations:
                elements = line.strip().split()
                if int(elements[0]) == class_id:
                    # Extract bounding box details
                    x_center, y_center, width, height = map(float, elements[1:])          
                    # Convert normalized coordinates to pixel values
                    image = cv2.imread(image_path)
                    img_height, img_width = image.shape[:2]
                    x1 = int((x_center - width / 2) * img_width)
                    y1 = int((y_center - height / 2) * img_height)
                    x2 = int((x_center + width / 2) * img_width)
                    y2 = int((y_center + height / 2) * img_height)
                        
                    # Extract the region of interest (ROI)
                    roi = image[y1:y2, x1:x2]

                    #if contains_blue_flag(roi):
                        # print(f"Contains blue flag? {image_path} ")
                        
                    # Check for strong contrast into darker blue
                    if not contains_blue_flag(roi):
                      #  if not os.path.exists(output_folder):
                      #  os.makedirs(output_folder)
                        #print(f"Beginning filtering {job} for non overlapping flags...")
    
                        # Move image and annotation to output folder
                        # shutil.copy(image_path, os.path.join(output_folder, os.path.basename(image_path)))
                        shutil.move(image_path, os.path.join(obj_train_new_data_path, os.path.basename(image_path)))
                        shutil.move(annotation_path, os.path.join(obj_train_new_data_path, os.path.basename(annotation_path)))
                        #shutil.move(image_path, os.path.join(obj_train_new_data_path, os.path.basename(image_path)))
                        #shutil.move(annotation_path, os.path.join(obj_train_new_data_path, os.path.basename(annotation_path)))
                        # print(f"Moved {os.path.basename(image_path)} to {output_folder}")
                        break
    print(f"Finished filtering {job}, ...")

def contains_blue_flag(roi):
    """
    Check if a region of interest contains a blue flag.
    :param roi: Region of interest (ROI) as a numpy array.
    :return: True if blue flag is detected, False otherwise.
    """
    # Convert ROI to HSV color space
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    # Define HSV range for darker blue
    #lower_blue = np.array([100, 150, 50])  
    #lower_blue = np.array([204, 20, 86])    
    # Photoshop, OpenCV use different terminologieS - HSB in PS underlying maths/scaling for HSB and HSV is same.
    upper_blue = np.array([130, 255, 200])
    lower_blue = np.array([90, 50, 50])  # Broaden range for darker/lighter blue
    upper_blue = np.array([140, 255, 255])
    # cde1f1 - try 207 15 94 = hsb? for job 121 image 000000.png 204 20 86 ?
    # Create a mask for blue regions
    blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)
    ## hex code for imageiin 121 is cde1f1
    # Detect edges within the blue mask
    edges = cv2.Canny(blue_mask, 100, 200)
    # Check if there are strong edges in the blue mask
    if np.any(edges):
        return True  # Blue flag detected
    return False  # No blue flag detected
    #print(f"No blue flag detected")

if __name__ == "__main__":
    base_folder = r'D:\FlagDetectionDatasets\ExportedDatasetsReduced'
    base_folder = 'D:/FlagDetectionDatasets/Augmentation/Cleaning_Demos' #Job_24'
    filter_non_encroaching_blue_flag_bb_flat(base_folder, 'Job_37_Aug_Illum', 1)   ## 1 is for the red_yellow flag 
    # filter_non_encroaching_blue_flag_bb(base_folder, 'Job_121', 1)   ## 1 is for the red_yellow flag 
    #filter_non_encroaching_blue_flag_bb(base_folder, 'Job_23', 0)   # Red flag 
    #filter_non_encroaching_blue_flag_bb_flat(base_folder, 'Job_24', 0)   # Red flag 
    #filter_non_encroaching_blue_flag_bb(base_folder, 'Job_23', 0) # red flag 
    #filter_non_encroaching_blue_flag_bb(base_folder, 'Job_23',0)
    #filter_non_encroaching_blue_flag_bb(base_folder, 'Job_24',0)
    #filter_non_encroaching_blue_flag_bb_flat(base_folder, 'Job_25',0) # 0 = Red flag  
    # filter_non_encroaching_blue_flag_bb(base_folder, 'Job_27', 0)
    #filter_non_encroaching_blue_flag_bb_flat(base_folder, 'Job_28', 0)
    #filter_non_encroaching_blue_flag_bb_flat(base_folder, 'Job_24', 0)

    #filter_non_encroaching_blue_flag_bb(base_folder, 'Job_28', 0)  # red flag 
    # filter_non_encroaching_blue_flag_bb(base_folder, 'Job_96', 0)  ## red flag 
    #filter_non_encroaching_blue_flag_bb(base_folder, 'Job_37', 0)  ## CHECK RESULT FOR jOB 37
    # filter_non_encroaching_blue_flag_bb(base_folder, 'Job_119', 1)   ## 1 red_yellow flag # 

    

Checking Job_37_Aug_Illum for flags where the blue flag bounding box does not overlap the red or red yellow flag in ...
Base folder D:/FlagDetectionDatasets/Augmentation/Cleaning_Demos...
Finished filtering Job_37_Aug_Illum, ...
